Please note that this tutorial is focused at case study.  If you are interested in more technical details, please consult the other tutorials.

# Requirements

## Memory

The code of the notebook requires at least 8.5 GB (8.0 GiB) of free memory.


## Environment

1. Anaconda Python distribution (tested with _Miniconda3-py39\_4.12.0-Linux-x86\_64.sh_, _conda v. 4.12.0_).
2. Jupyter server (see _extras/jupyter\_server.sh_ for details).
3. Anaconda environments (run _setup\_conda\_envs.sh_).
4. gmsh (not necessary if you already have meshes in either MSH or XDMF format).


## Setup

### Mesh for kESI

You need to have a mesh in XDMF format.  Run:

    conda activate kesi37
    cd extras/FEM/meshes
    snakemake meshes/four_spheres_csf_3_mm_plain/coarse.xdmf -j 1

It may take a while.  At least 1 GB (0.91 GiB) of free memory is necessary.


### Mesh for forward modelling

You will also need a mesh of slightly different geometry to simulate difference between assumed and actual brain.  Run:

    snakemake meshes/four_spheres_csf_1_mm_plain/normal.xdmf -j 1

At least 5.3 GB (5 GiB) of free memory is necessary.

#### Preprocessing

We are going to use:
  - the _extras/FEM/electrode\_locations/tutorial/case\_study.ini_ electrode positions,
  - the _extras/FEM/meshes/meshes/four\_spheres\_csf\_3\_mm\_plain/coarse.xdmf_ mesh, and
  - the _extras/FEM/model_properties/four\_spheres\_csf\_3\_mm.ini_ properties.

The preprocessing workflow is defined in the _extras/Snakefile_ and we run it with `snakemake`:

    cd extras
    snakemake .tutorial_case_study -j 1 --resources mem_mb=15000 --restart-times 2

That may take few **days**.  Ensure that you have enough diskspace (74+ GiB) at the filesystem of the _extras/FEM/solutions/tutorial/case\_study/_ directory.

> The `-j 1` parameter sets the number of jobs that can be run in parallel to 1.  If more CPU cores are available, you can increase that number to increase throughput, thus reduce the total preprocessing time.  But mind that in some systems (e.g. certain virtual machines) increase of paralellization beyond certain limit may significantly compromise performance.  It took about 10 days when run with `-j 2` on a dualcore virtual machine while only 6 days when run serially on a single core one.  The same physical machine was used in both cases.
> The `--resources mem_mb=15000` parameter tells the scheduler to possibly limit the total memory used by all simultaneous jobs to 15000 MiB.  You can adjust the parameter according to the value of the `-j` parameter and available free memory.  The scheduler limits the memory available for a job to 15000 MiB in case of calculating the leadfield correction, and to 7500 MiB in case of sampling the leadfield.  Both limits include about 2 GiB of safety margins.  Moreover, if a job fails, it is restarted (up to 2 times - given by the `--restart-times 2` parameter) with a doubled limit.

<!-- Solving requires at least 13.5 GB of memory, and sampling it - at least 5.7 GB -->

# FRR: Fast Reciprocal Reconstructor kernel construction tools 

The `_fast_reciprocal_reconstructor` module contains tools which allow for fast construction of (cross)kernels.  They use discrete for high throughput integration.

## Electrode object

An electrode object contains information about electrode spatial location (`.x`, `.y` and `.z` attribute), which is an absolute minimum to be used by kESI (in this case: kCSD with known base function in potential space).  It also provides additional information about:
- leadfield correction (`.correction_leadfield()` method) which enables kESI for setups violating kCSD assumptions,
  while facilitating application of analitically derived kCSD base functions to avoid significant numerical errors,
- base conductivity (`.base_conductivity` attribute) assumed when calculating the leadfield correction,
- grid used to sample the leadfield correction (`.SAMPLING_GRID` attribute).

In [ ]:
import numpy as np

class Electrode(object):
    def __init__(self, filename):
        """
        Parameters
        ----------
        
        filename : str
            Path to the sampled correction potential.
        """
        self.filename = filename
        with np.load(filename) as fh:
            self.SAMPLING_GRID = [fh[c] for c in 'XYZ']
            self.x, self.y, self.z = fh['LOCATION']
            self.base_conductivity = fh['BASE_CONDUCTIVITY']

    def correction_leadfield(self, X, Y, Z):
        """
        Correction of the leadfield of the electrode
        for violation of kCSD assumptions
        
        Parameters
        ----------
        X, Y, Z : np.array
            Coordinate matrices of the same shape.
        """
        with np.load(self.filename) as fh:
            return self._correction_leadfield(fh['CORRECTION_POTENTIAL'],
                                              [X, Y, Z])

    def _correction_leadfield(self, SAMPLES, XYZ):
        # if XYZ points are in nodes of the sampling grid,
        # no time-consuming interpolation is necessary
        return SAMPLES[self._sampling_grid_indices(XYZ)]

    def _sampling_grid_indices(self, XYZ):
        return tuple(np.searchsorted(GRID, COORD)
                     for GRID, COORD in zip(self.SAMPLING_GRID, XYZ))

In [ ]:
import glob

electrodes = [Electrode(filename)
              for filename in glob.glob('FEM/solutions/tutorial/case_study/sampled/9/*.npz')]

In [ ]:
len(electrodes)

## Model source

While FRR tools operate on base function profiles defined as callables, we can use convenience kCSD base function objects as model bases (bases which centroid is `(0, 0, 0)`).

In [ ]:
from _common_new import SphericalSplineSourceKCSD, GaussianSourceKCSD3D

SRC_R = 9.8e-3
BASE_CONDUCTIVITY = electrodes[0].base_conductivity

spline_nodes = [SRC_R / 3, SRC_R]
spline_polynomials = [[1],
                      [0,
                       6.75 / SRC_R,
                       -13.5 / SRC_R ** 2,
                       6.75 / SRC_R ** 3]]
model_src = SphericalSplineSourceKCSD(0, 0, 0,
                                      spline_nodes,
                                      spline_polynomials,
                                      BASE_CONDUCTIVITY)

## Convolver object

The convolver is the engine of the FRR tools.  It is used to:
- integrate leadfields weighted by a CSD profile,
- obtain CSD profile of a mixture of base functions.

The convolver operates on three regular 3D grids of coordinates:
- _POT_ grid used for leadfield (_reciprocal potential_) integration,
- _CSD_ grid used for CSD profile calculation,
- _SRC_ grid used for distributing of base function centroids.

The _SRC_ grid is an intersection (in set arithmetic sense) of the _POT_ and the _CSD_ grids, thus they define the convolver unequivocally.

As convolver will use the Romberg method for integration ($2^k + 1$ quadrature nodes), the integrated CSD profile will be cropped to $2^{k - 1} h$ from centroid in every dimension, where $h$ is the quadrature step size (distance between adjacent _POT_ grid nodes).  To avoid errors by design, we want $h \geq 2^{1 - k} R$.

In [ ]:
from _fast_reciprocal_reconstructor import Convolver

ROMBERG_K = 5

_h_min = SRC_R * 2**(1 - ROMBERG_K)

_pot_grid = [_A[::2] for _A in electrodes[0].SAMPLING_GRID]
_csd_grid = [_A[::2] for _A in _pot_grid]

convolver = Convolver(_pot_grid, _csd_grid)

for _h in convolver.steps('POT'):
    assert _h >= _h_min, f'{_h} < {_h_min}'

## Convolver interface

The convolver interface binds the convolver to:
- a CSD profile,
- weights of a quadrature of equally-spaced nodes,
- boolean mask of nodes of the _SRC_ grid with centroids of the base functions.

When analytical solution of the kCSD forward problem is used coupled with numeric integration of leadfield correction, it is advised not to put centroids near the boundary of the _SRC_ grid.

In [ ]:
from _fast_reciprocal_reconstructor import ConvolverInterfaceIndexed
from scipy.integrate import romb

ROMBERG_N = 2 ** ROMBERG_K + 1
ROMBERG_WEIGHTS = romb(np.identity(ROMBERG_N)) * 2 ** -ROMBERG_K
BRAIN_RADIUS = 0.079

SRC_MASK = ((convolver.SRC_Z > -0.06 + SRC_R)
            & (np.sqrt(np.square(convolver.SRC_X)
                       + np.square(convolver.SRC_Y)
                       + np.square(convolver.SRC_Z)) < BRAIN_RADIUS - SRC_R))

In [ ]:
print(SRC_MASK.sum())

In [ ]:
convolver_interface = ConvolverInterfaceIndexed(convolver,
                                                model_src.csd,
                                                ROMBERG_WEIGHTS,
                                                SRC_MASK)

## Potential At Electrode object

### Potential At Electrode: analytical solution of the forward problem (kCSD)

In [ ]:
from _fast_reciprocal_reconstructor import PAE_Analytical

In [ ]:
pae_kcsd = PAE_Analytical(convolver_interface,
                          potential=model_src.potential)

### Potential At Electrode: numerically corrected analytical solution of the forward problem (kESI)

In [ ]:
from _fast_reciprocal_reconstructor import PAE_AnalyticalCorrectedNumerically

In [ ]:
pae_kesi = PAE_AnalyticalCorrectedNumerically(convolver_interface,
                                              potential=model_src.potential)

## Kernel constructor and cross-kernel constructor

In [ ]:
from _fast_reciprocal_reconstructor import KernelConstructor, CrossKernelConstructor

In [ ]:
kernel_constructor = KernelConstructor()

In [ ]:
CSD_MASK = np.ones(convolver.shape('CSD'),
                   dtype=bool)

In [ ]:
kernel_constructor.create_crosskernel = CrossKernelConstructor(convolver_interface,
                                                               CSD_MASK)

As all elements of `CSD_MASK` are true, conversion of the reconstructed CSD vector is as simple as its rearrangement to match the _CSD_ grid of `convolver`.  We declare an auxilary `to_3D()` function for that purpose.

In [ ]:
def to_3D(CSD):
    return CSD.reshape(convolver.shape('CSD'))

# kCSD reconstructor

## Construction of kernels

In [ ]:
%%time
PHI_KCSD = kernel_constructor.create_base_images_at_electrodes(electrodes,
                                                               pae_kcsd)

In [ ]:
%%time
KERNEL_KCSD = kernel_constructor.create_kernel(PHI_KCSD)

In [ ]:
%%time
CROSSKERNEL_KCSD = kernel_constructor.create_crosskernel(PHI_KCSD)

It may take more than half a minute.

In [ ]:
del PHI_KCSD  # the array is large and no longer needed

## Reconstructor object

In [ ]:
from kesi._verbose import _CrossKernelReconstructor as Reconstructor
from kesi._engine import _LinearKernelSolver as KernelSolver

In [ ]:
reconstructor_kcsd = Reconstructor(KernelSolver(KERNEL_KCSD),
                                   CROSSKERNEL_KCSD)

# Visualisation

For data visualisation we define an auxilary class `CardinalPlaneVisualisation`.

In [ ]:
import matplotlib.pyplot as plt
import cbf

class CardinalPlaneVisualisation(object):
    SPHERE_RADII = [0.079, 0.082, 0.086, 0.090]
    SPHERE_RADII_GT = [0.079, 0.080, 0.085, 0.090]

    def __init__(self,
                 grid,
                 plane_intersection,
                 dpi=35,
                 cmap=cbf.bwr,
                 amp=None,
                 length_factor=1,
                 length_unit='$m$',
                 unit_factor=1,
                 unit=''):
        self.grid = grid
        self.plane_intersection = np.array(plane_intersection)
        self.indices = [np.searchsorted(g, a)
                        for a, g in zip(plane_intersection,
                                        grid)]
        self.dpi = dpi
        self.cmap = cmap
        self.amp = amp
        self.length_factor = length_factor
        self.length_unit = length_unit
        self.unit_factor = unit_factor
        self.unit = unit

    def start_new_image(self, title, wx, wy, wz):
        self.fig = plt.figure(figsize=((wx + wy) / self.dpi,
                                       (wz + wy) / self.dpi))
        if title is not None:
            self.fig.suptitle(title)

        gs = plt.GridSpec(2, 2,
                          figure=self.fig,
                          width_ratios=[wx, wy],
                          height_ratios=[wz, wy])

        self.ax_xz = self.fig.add_subplot(gs[0, 0])
        self.ax_xz.set_aspect('equal')
        self.ax_xz.set_ylabel(f'Z [{self.length_unit}]')
        self.ax_xz.set_xlabel(f'X [{self.length_unit}]')

        self.ax_yx = self.fig.add_subplot(gs[1, 1])
        self.ax_yx.set_aspect('equal')
        self.ax_yx.set_ylabel(f'X [{self.length_unit}]')
        self.ax_yx.set_xlabel(f'Y [{self.length_unit}]')

        self.ax_yz = self.fig.add_subplot(gs[0, 1],
                                          sharey=self.ax_xz,
                                          sharex=self.ax_yx)
        self.ax_yz.set_aspect('equal')

        self.cax = self.fig.add_subplot(gs[1, 0])
        self.cax.set_visible(False)

    def finish_image(self):
        x, y, z = self.length_factor * self.plane_intersection

        self.ax_xz.axvline(x, ls=':', color=cbf.BLACK)
        self.ax_xz.axhline(z, ls=':', color=cbf.BLACK)

        self.ax_yx.axvline(y, ls=':', color=cbf.BLACK)
        self.ax_yx.axhline(x, ls=':', color=cbf.BLACK)

        self.ax_yz.axvline(y, ls=':', color=cbf.BLACK)
        self.ax_yz.axhline(z, ls=':', color=cbf.BLACK)
        self.fig.colorbar(self.im, ax=self.cax,
                          orientation='horizontal',
                          label=self.unit)

    def plot_volume(self, DATA, title=None, amp=None):
        self.start_new_image(title, *DATA.shape)
        ix, iy, iz = self.indices
        self._plot_planes([DATA[ix:ix+1, :, :],
                           DATA[:, iy:iy+1, :],
                           DATA[:, :, iz:iz+1],
                           ],
                           amp if amp is not None else abs(DATA).max())
        self.finish_image()

    def _plot_planes(self, DATA_PLANES, amp):
        DATA_ZY = DATA_PLANES[0][0, :, :].T * self.unit_factor
        DATA_ZX = DATA_PLANES[1][:, 0, :].T * self.unit_factor
        DATA_XY = DATA_PLANES[2][:, :, 0] * self.unit_factor
        
        def _extent(first, second):
            _first = self.grid[first] * self.length_factor
            _second = self.grid[second] * self.length_factor
            return (_first.min(), _first.max(),
                    _second.min(), _second.max())

        self.ax_xz.imshow(DATA_ZX,
                          vmin=-amp * self.unit_factor,
                          vmax=amp * self.unit_factor,
                          cmap=self.cmap,
                          origin='lower',
                          extent=_extent(0, 2))
        self.ax_yx.imshow(DATA_XY,
                          vmin=-amp * self.unit_factor,
                          vmax=amp * self.unit_factor,
                          cmap=self.cmap,
                          origin='lower',
                          extent=_extent(1, 0))
        self.im = self.ax_yz.imshow(DATA_ZY,
                                    vmin=-amp * self.unit_factor,
                                    vmax=amp * self.unit_factor,
                                    cmap=self.cmap,
                                    origin='lower',
                                    extent=_extent(1, 2))

    def plot_planes(self,
                    DATA_PLANES,
                    title=None,
                    amp=None):

        DATA_YZ, DATA_XZ, DATA_XY = DATA_PLANES
        wx, wy, _ = DATA_XY.shape
        wz = DATA_YZ.shape[2]
        assert DATA_YZ.shape[1] == wy
        assert DATA_XZ.shape[0] == wx
        assert DATA_XZ.shape[2] == wz
        
        self.start_new_image(title, wx, wy, wz)
        self._plot_planes(DATA_PLANES,
                          amp if amp is not None else max(abs(_A).max() for _A in DATA_PLANES))
        self.finish_image()

    def compare_with_gt(self, GT, CSD, title=''):
        ERROR = CSD - GT
        error_L2 = np.sqrt(np.square(ERROR).sum() / np.square(GT_CSD).sum())
        amp = max(abs(CSD).max(),
                  abs(GT).max(),
                  abs(ERROR).max())
        self.plot_volume(GT,
                         title='GT CSD',
                         amp=amp)
        self._add_spheres(self.SPHERE_RADII_GT)
        self.plot_volume(CSD,
                         title=f'{title} reconstruction',
                         amp=amp)
        self._add_spheres(self.SPHERE_RADII)
        self.plot_volume(ERROR,
                         title=f'{title} error (GT normalized L2 norm: {error_L2:.2g})',
                         amp=amp)

    def _add_spheres(self, sphere_radii):
        for c, ax in zip(self.plane_intersection,
                         [self.ax_yz,
                          self.ax_xz,
                          self.ax_yx]):
            for r2 in np.square(sphere_radii):
                self._plot_circle(ax, np.sqrt(r2 - np.square(c)))

    def _plot_circle(self, ax, r):
        ax.add_artist(plt.Circle((0, 0), r * self.length_factor,
                                facecolor='none',
                                edgecolor=cbf.BLACK,
                                linestyle=':'))

    @property
    def PLANES_XYZ(self):
        return [[[c] if i == j else A for j, A in enumerate(self.grid)]
                for i, c in enumerate(self.plane_intersection)]

In [ ]:
csd_grid = [_x.flatten() for _x in convolver.CSD_GRID]

In [ ]:
csd_plotter = CardinalPlaneVisualisation(csd_grid,
                                         [-0.05, 0.015, -0.02],
                                         unit_factor=1e-3,
                                         unit='$\\frac{pA}{mm^3}$',
                                         length_factor=1e3,
                                         length_unit='$mm$')

# Ground truth CSD and its potential at the electrodes

We derive GT CSD as an eigensource (scaled by a factor) of kCSD.  The order of the returned eigenvalues (and eigenvectors) is reversed for the sake of the highest-first convention.

## GT CSD

In [ ]:
%%time
EIGENVALUES_KCSD, EIGENVECTORS_KCSD = np.linalg.eigh(KERNEL_KCSD)
EIGENVALUES_KCSD, EIGENVECTORS_KCSD = EIGENVALUES_KCSD[::-1], EIGENVECTORS_KCSD[:, ::-1]

GT_CSD = to_3D(reconstructor_kcsd(EIGENVECTORS_KCSD[:, 2] * 100))

In [ ]:
csd_plotter.plot_volume(GT_CSD, 'GT CSD')

## FEM forward modelling

In [ ]:
import configparser
import dolfin
import scipy.interpolate as si

import FEM.fem_common as fc

We define a FEM forward model. It is a callable, which accepts CSD profile as a callable compatible with `scipy.interpolate.RegularGridInterpolator`.

In [ ]:
class ForwardModel(object):
    GROUNDED_PLATE_AT = -0.088

    def __init__(self, mesh, degree, config):
        self.fm = fc.FunctionManager(mesh, degree, 'CG')
        self.config = configparser.ConfigParser()
        self.config.read(config)
        
        self.V = self.fm.function_space
        mesh = self.fm.mesh

        n = self.V.dim()
        d = mesh.geometry().dim()

        self.dof_coords = self.V.tabulate_dof_coordinates()
        self.dof_coords.resize((n, d))
        
        self.csd_f = self.fm.function()
        
        self.subdomains = self.fm.load_subdomains()
        self.dx = dolfin.Measure("dx")(subdomain_data=self.subdomains)

    @property
    def CONDUCTIVITY(self):
        for section in self.config.sections():
            if self._is_conductive_volume(section):
                yield (self.config.getint(section, 'volume'),
                       self.config.getfloat(section, 'conductivity'))

    def _is_conductive_volume(self, section):
        return (self.config.has_option(section, 'volume')
                and self.config.has_option(section, 'conductivity')) 
        
    def __call__(self, csd_interpolator):
        self.csd_f.vector()[:] = csd_interpolator(self.dof_coords)
        
        dirichlet_bc_gt = dolfin.DirichletBC(self.V,
                                     dolfin.Constant(0),
                                     (lambda x, on_boundary:
                                      on_boundary and x[2] <= self.GROUNDED_PLATE_AT))
        test = self.fm.test_function()
        trial = self.fm.trial_function()
        potential = self.fm.function()
        
        
        dx = self.dx
        a = sum(dolfin.Constant(c)
                * dolfin.inner(dolfin.grad(trial),
                               dolfin.grad(test))
                * dx(i)
                for i, c
                in self.CONDUCTIVITY)
        L = self.csd_f * test * dx
        
        b = dolfin.assemble(L)
        A = dolfin.assemble(a)
        dirichlet_bc_gt.apply(A, b)
        
        solver = dolfin.KrylovSolver("cg", "ilu")
        solver.parameters["maximum_iterations"] = 10000
        solver.parameters["absolute_tolerance"] = 1E-8
        solver.solve(A, potential.vector(), b)
        
        return potential

To simulate the differences between actual and assumed head geometries, we use model with thinner CSF layer for forward modelling.

In [ ]:
%%time
fem = ForwardModel(mesh='FEM/meshes/meshes/four_spheres_csf_1_mm_plain/normal.xdmf',
                   degree=1,
                   config='FEM/model_properties/four_spheres_csf_1_mm.ini')

It may take longer than 2 minutes.

We simulate the potential generated by the ground truth CSD profile.

In [ ]:
%%time
_csd = si.RegularGridInterpolator(csd_grid,
                                  GT_CSD,
                                  bounds_error=False,
                                  fill_value=0)
potential = fem(_csd)

del _csd  # the object is large and no longer needed

It may take longer than 2 minutes.

We probe the potential in the cardinal planes to visualise it.

In [ ]:
potential_plotter = CardinalPlaneVisualisation(csd_grid,
                                               [-0.05, 0.015, -0.02],
                                               unit='$\mu{}V$',
                                               length_factor=1e3,
                                               length_unit='$mm$',
                                               cmap=cbf.PRGn)

In [ ]:
%%time
V_PLANES = []

for _X, _Y, _Z in potential_plotter.PLANES_XYZ:
    with np.nditer([np.reshape(_X, (-1, 1, 1)),
                    np.reshape(_Y, (1, -1, 1)),
                    np.reshape(_Z,  (1, 1, -1)),
                    None]) as it:
        for _x, _y, _z, _res in it:
            try:
                _res[...] = potential(_x, _y, _z)
            except RuntimeError:
                _res[...] = np.nan

        V_PLANES.append(np.ma.masked_invalid(it.operands[3]))

It may take more than a minute.

In [ ]:
potential_plotter.plot_planes(V_PLANES, 'POTENTIAL')

We probe the potential at electrodes.

In [ ]:
GT_V = np.array([potential(_e.x, _e.y, _e.z) for _e in electrodes])

In [ ]:
del potential, fem  # these objects are large and no longer needed

To make the case more realistic we introduce 1% noise.

In [ ]:
np.random.seed(42)
NOISE_V = np.random.normal(loc=GT_V,
                           scale=0.01 * GT_V.std())

# Reconstruction

## kCSD

As the ground truth CSD is a kCSD eigensource, kCSD is (theoretically) able to reconstruct it exactly.  But the forward model does not conform to assumptions of kCSD, which may lead to reconstruction artifacts.

In [ ]:
CSD_KCSD = to_3D(reconstructor_kcsd(NOISE_V))

In [ ]:
csd_plotter.compare_with_gt(GT_CSD,
                            CSD_KCSD,
                            'kCSD')

As you see, the reconstruction is mostly error.

In [ ]:
del CSD_KCSD  # the array is large and no longer needed

## Regularization

### Cross-validation

To counter the noise we select a regularization parameter which minimizes cross-validation error.  The span of parameters from which it is being selected should cover eigenvalues of the kernel.

In [ ]:
plt.figure(figsize=(18, 12))
plt.plot(EIGENVALUES_KCSD)
plt.yscale('log')

In [ ]:
REGULARIZATION_PARAMETERS = np.logspace(-2, 15, 17 * 10 + 1)

In [ ]:
from _common_new import cv

In [ ]:
%%time
CV_ERRORS_KCSD = cv(reconstructor_kcsd, NOISE_V, REGULARIZATION_PARAMETERS)

It may take more than two hours if other tasks are running.  Otherwise it may take less than a minute.

In [ ]:
regularization_parameter_kcsd = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KCSD)]

In [ ]:
plt.plot(REGULARIZATION_PARAMETERS,
         CV_ERRORS_KCSD,
         color=cbf.BLUE)
plt.axvline(regularization_parameter_kcsd,
            ls=':',
            color=cbf.BLUE)
plt.xscale('log')
plt.xlabel('regularization parameter')
plt.yscale('log')
plt.ylabel('L2 norm of cross-validation error')

## Regularized kCSD

In [ ]:
CSD_KCSD_CV = to_3D(reconstructor_kcsd(NOISE_V, regularization_parameter_kcsd))

In [ ]:
csd_plotter.compare_with_gt(GT_CSD,
                            CSD_KCSD_CV,
                            'kCSD (regularized)')

The reconstruction still is mostly error.

In [ ]:
del CSD_KCSD_CV  # the array is large and no longer needed

In [ ]:
del reconstructor_kcsd, CROSSKERNEL_KCSD  # These objects are large and no longer needed

## Unregularized kESI

kESI accounts for violation of kCSD assumptions, thus we expect more reliable reconstruction.

In [ ]:
%%time
PHI_KESI = kernel_constructor.create_base_images_at_electrodes(electrodes,
                                                               pae_kesi)

It may take more than 20 minutes.

In [ ]:
%%time
KERNEL_KESI = kernel_constructor.create_kernel(PHI_KESI)

In [ ]:
%%time
CROSSKERNEL_KESI = kernel_constructor.create_crosskernel(PHI_KESI)

It may take more than half a minute.

In [ ]:
del PHI_KESI  # the array is large and no longer needed

In [ ]:
reconstructor_kesi = Reconstructor(KernelSolver(KERNEL_KESI),
                                   CROSSKERNEL_KESI)

In [ ]:
CSD_KESI = to_3D(reconstructor_kesi(NOISE_V))

In [ ]:
csd_plotter.compare_with_gt(GT_CSD,
                            CSD_KESI,
                            'kESI')

As you see, the unregularized kESI reconstruction somehow resembles the ground truth.

In [ ]:
del CSD_KESI  # the array is large and no longer needed

## Regularized kESI

In [ ]:
EIGENVALUES_KESI = np.linalg.eigvalsh(KERNEL_KESI)[::-1]

We reversed order of the returned eigenvalues for the sake of the highest-first convention.

In [ ]:
plt.figure(figsize=(18, 12))
plt.plot(EIGENVALUES_KCSD, label='kCSD', ls='-')
plt.plot(EIGENVALUES_KESI, label='kESI', ls=':')
plt.yscale('log')
plt.legend(loc='best')

In [ ]:
%%time
CV_ERRORS_KESI = cv(reconstructor_kesi, NOISE_V, REGULARIZATION_PARAMETERS)

It may take more than 2 hours if other tasks are running.  Otherwise it may take less than a minute.

In [ ]:
regularization_parameter_kesi = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KESI)]

In [ ]:
plt.plot(REGULARIZATION_PARAMETERS,
         CV_ERRORS_KCSD,
         color=cbf.BLUE,
         label='kCSD')
plt.plot(REGULARIZATION_PARAMETERS,
         CV_ERRORS_KESI,
         color=cbf.VERMILION,
         label='kESI')
plt.axvline(regularization_parameter_kcsd,
            ls=':',
            color=cbf.BLUE)
plt.axvline(regularization_parameter_kesi,
            ls=':',
            color=cbf.VERMILION)
plt.xscale('log')
plt.xlabel('regularization parameter')
plt.yscale('log')
plt.ylabel('L2 norm of cross-validation error')
plt.legend(loc='best')

In [ ]:
CSD_KESI_CV = to_3D(reconstructor_kesi(NOISE_V, regularization_parameter_kesi))

In [ ]:
csd_plotter.compare_with_gt(GT_CSD,
                            CSD_KESI_CV,
                            'kESI (regularized)')

The kESI reconstruction definitely benefitted from regularization.

In [ ]:
del CSD_KESI_CV  # the array is large and no longer needed